# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.

Read more about planner [here](https://aka.ms/sk/concepts/planner)


In [14]:
!pip install semantic-kernel==v0.9.3b1

In [15]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

In [16]:
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments

In [17]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai

kernel = sk.Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )

## It all begins with an ask


In [18]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing plugins to the planner

The planner needs to know what plugins are available to it. Here we'll give it access to the `SummarizePlugin` and `WriterPlugin` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset.

You can also include native functions as well. Here we'll add the TextPlugin.


In [19]:
from semantic_kernel.core_plugins.text_plugin import TextPlugin

plugins_directory = "../../samples/plugins/"
summarize_plugin = kernel.import_plugin_from_prompt_directory(plugins_directory, "SummarizePlugin")
writer_plugin = kernel.import_plugin_from_prompt_directory(plugins_directory, "WriterPlugin")
text_plugin = kernel.import_plugin_from_object(TextPlugin(), "TextPlugin")

Define your ASK. What do you want the Kernel to do?


# Basic Planner


Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.


In [20]:
from semantic_kernel.planners.basic_planner import BasicPlanner

planner = BasicPlanner(service_id)

In [21]:
basic_plan = await planner.create_plan(ask, kernel)

In [22]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterPlugin.Brainstorm"},
        {"function": "WriterPlugin.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterPlugin.Translate", "args": {"language": "French"}},
        {"function": "TextPlugin.uppercase"}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the plugins that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.


Let's also define an inline plugin and have it be available to the Planner. Be sure to give it a function name and plugin name.


In [23]:
prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""

exec_settings = sk_oai.OpenAIChatPromptExecutionSettings(
    service_id=service_id,
    max_tokens=2000,
    temperature=0.8,
)

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    function_name="shakespeare",
    plugin_name="ShakespearePlugin",
    name="planner",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=exec_settings,
)

shakespeare_plugin = kernel.create_function_from_prompt(
    function_name="shakespeare",
    plugin_name="ShakespearePlugin",
    prompt_template_config=prompt_template_config,
)

Let's update our ask using this new plugin


In [24]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

# TODO: we cannot add an updated ask to a current plan because the underlying plugins already exist
kernel = sk.Kernel()
service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )

plugins_directory = "../../samples/plugins/"
summarize_plugin = kernel.import_plugin_from_prompt_directory(plugins_directory, "SummarizePlugin")
writer_plugin = kernel.import_plugin_from_prompt_directory(plugins_directory, "WriterPlugin")
text_plugin = kernel.import_plugin_from_object(TextPlugin(), "TextPlugin")

planner = BasicPlanner(service_id)
new_plan = await planner.create_plan(ask, kernel, prompt)

Variable `$: input` not found in the KernelArguments


In [25]:
print(new_plan.generated_plan)

Hark! An humble request doth reach mine ears,
To recast the prior words with a Shakespearean air.
With quill in hand, and inkwell by my side,
I embark upon this task, undaunted and wide.

Pray, let me conjure verses befitting the Bard,
To rewrite the lines, as thou hast asked hard.
In lofty language and majestic tone,
I shall transform the words, to make them thine own.

"Listen, good people! A humble plea I hear,
To render the former speech in Shakespearean cheer.
With feathered pen poised, and inkpot at hand,
I embark on this task, across vast and grand.

Allow me to conjure verses, worthy of the Bard's art,
To rewrite the lines, as you have asked, impart.
In lofty words and majestic voice,
I shall transform these phrases, to make them your choice."


### Executing the plan


Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.


In [26]:
results = await planner.execute_plan(new_plan, kernel)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [27]:
print(results)

NameError: name 'results' is not defined

# The Plan Object Model


To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)


# Sequential Planner


The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Plugins/SequentialPlanning/skprompt.txt)


In [ ]:
from semantic_kernel.planners import SequentialPlanner

planner = SequentialPlanner(kernel, service_id)

In [ ]:
sequential_plan = await planner.create_plan(goal=ask)

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions


In [ ]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Let's ask the sequential planner to execute the plan.


In [ ]:
result = await sequential_plan.invoke(kernel)

In [ ]:
print(result)

# Action Planner


The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.


In [ ]:
from semantic_kernel.planners import ActionPlanner

planner = ActionPlanner(kernel, service_id)

Let's add more plugins to the kernel


In [ ]:
from semantic_kernel.core_plugins import (
    MathPlugin,
    TextPlugin,
    TimePlugin,
)

kernel.import_plugin_from_object(MathPlugin(), "math")
kernel.import_plugin_from_object(TimePlugin(), "time")
kernel.import_plugin_from_object(TextPlugin(), "text")

In [ ]:
ask = "What is the sum of 110 and 990?"

In [ ]:
plan = await planner.create_plan(goal=ask)

In [ ]:
result = await plan.invoke(kernel)

In [ ]:
print(result)

# Stepwise Planner


Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)


Let's create a Bing Search native plugin that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)


In [ ]:
from semantic_kernel.connectors.search_engine import BingConnector
from semantic_kernel.core_plugins import WebSearchEnginePlugin

BING_API_KEY = sk.bing_search_settings_from_dot_env()
connector = BingConnector(BING_API_KEY)
kernel.import_plugin_from_object(WebSearchEnginePlugin(connector), plugin_name="WebSearch")

Let's also add a couple more plugins


In [ ]:
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin

kernel.import_plugin_from_object(TimePlugin(), "time")
kernel.import_plugin_from_object(MathPlugin(), "math")

In [ ]:
from semantic_kernel.planners.stepwise_planner import StepwisePlanner, StepwisePlannerConfig

planner = StepwisePlanner(kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000))

Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.


In [ ]:
ask = """How many total championships combined do the top 5 teams in the NBA have?"""

plan = planner.create_plan(goal=ask)

In [ ]:
result = await plan.invoke()

In [ ]:
print(result)

Let's see the steps that the AI took to get to the answer.


In [ ]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:", step.description)
    print("Function:", step.plugin_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print("  Output:\n", str.replace(result[step._outputs[0]], "\n", "\n  "))